In [3]:
%run "../Get Features.ipynb"

In [4]:
import numpy as np

In [5]:
%%time
a = get_all()
a = [np.hstack(i) for i in a]
pronoun, verbs, entity_counts, sen_len, sen_pos, stop_counts, output = a

loaded tags
loaded doc metadata
loaded stop counts
loaded outputs
CPU times: user 2min 29s, sys: 49 s, total: 3min 18s
Wall time: 5min 2s


In [6]:
output = [0 if x == 2 else x for x in output]

In [8]:
import torch
import torch.nn as nn
import torch.nn.functional as f
import torch.optim as optim

The history saving thread hit an unexpected error (OperationalError('database or disk is full',)).History will not be written to the database.


In [9]:
from model import LogisticRegression

In [10]:
input = torch.t(torch.FloatTensor(a[:-1]))
output = torch.Tensor(output).unsqueeze(1)

In [11]:
model = LogisticRegression()

In [12]:
LOSS = nn.BCEWithLogitsLoss()
OPTIM = optim.Adam(model.parameters())

In [13]:
NO_ITERATIONS = 10
BATCH_SIZE = 20

In [14]:
LOSS(model(input), output)

tensor(1.6919, grad_fn=<MeanBackward1>)

In [15]:
for i in range(NO_ITERATIONS):
    for j in range(0, len(input), BATCH_SIZE):
        model_output = model(input[j: j+BATCH_SIZE])
        truth = output[j: j+BATCH_SIZE]
        loss = LOSS(model_output, truth)
        model.zero_grad()
        loss.backward()
        OPTIM.step()
        print(j, end='\r')

In [16]:
LOSS(model(input), output)

tensor(0.5692, grad_fn=<MeanBackward1>)

In [17]:
model.weights.weight

Parameter containing:
tensor([[-0.0271,  0.0206,  0.2536,  0.0694, -0.0416, -0.0860]], requires_grad=True)

In [18]:
torch.save(model.state_dict(), 'log_reg_weights')